In [ ]:
#import opencv
import cv2
import pandas as pd
print(cv2.__version__)

In [2]:
# specify video path
video_path = '../video/football.mp4'
output_path = '../output/'

In [3]:
# create a dictionary of all sinhle object trackers in opencv
tracker_options = {'csrt': cv2.TrackerCSRT_create,
                   'kcf': cv2.TrackerKCF_create,
                   'mil': cv2.TrackerMIL_create,
                   'medianflow': cv2.TrackerMedianFlow_create,
                   'boosting': cv2.TrackerBoosting_create,
                   'mosse': cv2.TrackerMOSSE_create,
                   'tld': cv2.TrackerTLD_create
           }

In [4]:
# choose trackers
trackers = cv2.MultiTracker_create()

In [5]:
# input the video
video = cv2.VideoCapture(video_path)

In [6]:
# # For webcam
# video = cv2.VideoCapture(0)

In [7]:
# read first frame and draw bounding box around objects of interest
ret, frame = video.read()

In [8]:
frame_num = 1
num_objects = 3
df = pd.DataFrame()
for i in range(num_objects):
    cv2.imshow('Frame', frame)
    roi = cv2.selectROI('Frame', frame)
    tracker_i = tracker_options['csrt']()
    trackers.add(tracker_i,frame,roi)
    df = df.append(pd.DataFrame({'frame': frame_num,'object':i, 'x': roi[0],'y':roi[1],'w':roi[2],'h':roi[3]} ,index=[0]),ignore_index = True)
cv2.destroyAllWindows()


In [9]:
# update the tracker for all frames and draw new bounding boxes
while True:
    frame_num += 1
    ret, frame = video.read()
    if not ret:
        break
    (success, bboxes) = trackers.update(frame)
    for i, bbox in enumerate(bboxes):
        df = df.append(pd.DataFrame({'frame': frame_num,'object':i, 'x': bbox[0],'y':bbox[1],'w':bbox[2],'h':bbox[3]} ,index=[0]),ignore_index = True)
        (x,y,w,h) = [int(a) for a in bbox]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
    cv2.imshow('Frame',frame)
    key = cv2.waitKey(5) & 0xFF
    if key == ord('q'):
        break
video.release()
cv2.destroyAllWindows()

In [11]:
df.to_csv(output_path+'MOT.csv',header=True, index = False)